In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 단파장 비율 학습 및 테스트 (편차 분포)

# import_data = np.genfromtxt('./../swr_ratio_classification_test.csv', delimiter=',', dtype='float')
import_data = np.genfromtxt('./../rain_data_ratio_classification_test.csv', delimiter=',', dtype='float')

x_data = []
for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][0])
    x_data.append(temp)
    
# scaler = MinMaxScaler(feature_range=(0, 1))
# x_data = scaler.fit_transform(x_data)
test_x = x_data[85:]
x_data = x_data[:85]

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][1])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][1])
    test_y.append(temp)


y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 1])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

# W = tf.Variable(tf.random_normal([1, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[1, nb_classes], initializer=xavier_init(1, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(16001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        print(a[0][0], ', ', a[0][1], ', ', a[0][2], ', ', a[0][3], ', ', a[0][4])
#         if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
#             accuracy += 1
#     print("accuracy = ", float(accuracy / len(test_x) * 100))

0 24.829124
1000 1.2056425
2000 1.0954796
3000 0.98352087
4000 0.8780826
5000 0.782894
6000 0.6998503
7000 0.6290564
8000 0.5692323
9000 0.51882035
10000 0.47614273
11000 0.4395261
12000 0.40770137
13000 0.3799118
14000 0.35540086
15000 0.33357534
16000 0.31398845
-----------------------------
train_data =  85 test_data =  1
[3] [3.0]
1.3545464e-07 ,  0.00023597332 ,  0.25994316 ,  0.5282267 ,  0.21159405
